## AG-UI와 에이전트 프레임워크 통합

[AG-UI](https://docs.ag-ui.com/introduction)는 실시간 스트리밍, 상태 관리, 대화형 UI 구성 요소와 같은 고급 기능을 갖춘 웹 기반 AI 에이전트 애플리케이션을 구축할 수 있도록 지원하는 프로토콜입니다. 에이전트 프레임워크의 AG-UI 통합은 에이전트와 웹 클라이언트 간의 원활한 연결을 제공합니다.

### AG-UI란 무엇인가요?  
AG-UI는 AI 에이전트 인터페이스 구축을 위한 표준화된 프로토콜로 다음과 같은 기능을 제공합니다. 압축해서 간단하게 말하자면, AG-UI는 AI 에이전트와 클라이언트 애플리케이션 간의 통신을 관리하는 역할을 합니다. 예를 들면, Agent를 HTTP 서버로 노출 및 통신하는 역할을 제공하고, 클라이언트에서 HTTP를 통해 에이전트와 상호작용할 수 있도록 합니다. AG-UI는 Microsoft Agent Framework, OpenAI, LangGraph, Ollama, Mastra 등 주요 백엔드 시스템과 통합이 가능하며, CopilotKit의 React 컴포넌트와도 자연스럽게 호환됩니다         

- 원격 에이전트 호스팅 : 여러 클라이언트가 액세스할 수 있는 웹 서비스로 AI 에이전트를 배포합니다.
- 실시간 스트리밍 : 서버 전송 이벤트(SSE)를 사용하여 에이전트 응답을 스트리밍하여 즉각적인 피드백을 제공합니다.
- 표준화된 통신 : 신뢰할 수 있는 상담원 상호 작용을 위한 일관된 메시지 형식
- 세션 관리 : 여러 요청에 걸쳐 대화 맥락을 유지합니다.
- 고급 기능 : 사람 참여 승인, 상태 동기화 및 사용자 지정 UI 렌더링

### AG-UI는 언제 사용해야 할까요?  
다음과 같은 상황에서 AG-UI 사용을 고려해 보세요.

- AI 에이전트와 상호 작용하는 클라이언트 웹 또는 모바일 애플리케이션을 개발하는 경우  
- 에이전트를 여러 사용자가 동시에 접근할 수 있는 **서비스**로 배포하는 경우  
- 스트림 에이전트는 실시간으로 응답하여 사용자에게 즉각적인 피드백을 제공하는 경우
- 사용자가 실행 전에 작업을 확인하는 승인 워크플로우를 구현하는 경우  
- 상호작용적인 경험을 위해 클라이언트와 서버 간의 상태를 동기화하는 경우  
- 에이전트 도구 호출에 따라 사용자 지정 UI 구성 요소를 렌더링하는 경우  

우선 필요한 라이브러리를 설치합니다. 일반적으로는 MAF를 설치하였다면 이미 설치되어 있을 것입니다.

In [ ]:
pip install agent-framework-ag-ui --pre

올바로 설치되었는지 확인합니다.

In [ ]:
# agent_framework 패키지의 실제 구조 확인
import agent_framework_ag_ui

print("=== agent-framework-ag-ui 모듈 확인 ===")
print(f"agent-framework-ag-ui 버전: {agent_framework_ag_ui.__version__ if hasattr(agent_framework_ag_ui, '__version__') else 'N/A'}")
print(f"\nagent-framework-ag-ui 모듈 내용:")
print([x for x in dir(agent_framework_ag_ui) if not x.startswith('_')])

### Agent를 서버로 구성하기

다음의 코드를 살펴본 뒤, 이를 사용해서 agent_server.py 파일을 만듭니다(현재 폴더에 이미 파일이 제공되고 있습니다)

**서버 코드의 핵심 개념**
- add_agent_framework_fastapi_endpoint() : fastapi와 agent를 연결하여 AG-UI 엔드포인트를 등록합니다. 이를 통해서 자동 요청/응답 처리 및 SSE 스트리밍 기능을 제공합니다. 
- ChatAgent: 요청을 처리할 에이전트 프레임워크 에이전트
- FastAPI 통합 : FastAPI의 기본 비동기 지원 기능을 활용하여 응답 스트리밍을 제공합니다.
- 지침(instructions) : 에이전트는 기본 지침에 따라 생성되며, 클라이언트 메시지를 통해 해당 지침을 재정의할 수 있습니다.
- 구성 : AzureOpenAIChatClient는 환경 변수를 읽거나 지정된 매개변수를 직접 사용합니다.

In [ ]:
"""AG-UI server example."""
## 노트북에서 실행하지 마세요. 
## 터미널에서 python agent_server.py 명령으로 실행하세요.

import os
import uvicorn
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient
from agent_framework_ag_ui import add_agent_framework_fastapi_endpoint
from azure.identity import AzureCliCredential
from fastapi import FastAPI

# Read required configuration
endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
deployment_name = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

if not endpoint:
    raise ValueError("AZURE_OPENAI_ENDPOINT environment variable is required")
if not deployment_name:
    raise ValueError("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME environment variable is required")

# Azure OpenAI Chat Client 생성
chat_client = AzureOpenAIChatClient(
    credential=AzureCliCredential(),
    endpoint=endpoint,
    deployment_name=deployment_name,
)

# AzureOpenAIChatClient를 사용하는 기본적인 AI agent 생성
agent = ChatAgent(
    name="AGUIAssistant",
    instructions="당신은 도움이 되는 어시스턴트입니다. 🤖✨",
    chat_client=chat_client,
)

# Create FastAPI app
app = FastAPI(title="AG-UI Server")

# AG-UI endpoint 등록 ("/")
add_agent_framework_fastapi_endpoint(app, agent, "/")

if __name__ == "__main__":

    uvicorn.run(app, host="127.0.0.1", port=8888)

터미널을 새로 열고 다음과 같이 명령을 실행하여 서버를 시작합니다.

``` powershell
    cd .\AG-UI\
    python agent_server.py
```

그러면 다음과 같이 서버가 시작됩니다.

``` powershell
    (.venv) PS C:\GitHub\MAF_Workshop\AG-UI> python agent_server.py
    INFO:     Started server process [47172]
    INFO:     Waiting for application startup.
    INFO:     Application startup complete.
    INFO:     Uvicorn running on http://127.0.0.1:8888 (Press CTRL+C to quit)
    INFO:     127.0.0.1:62994 - "POST / HTTP/1.1" 200 OK
```

### 서버 테스트하기

서버가 실행 중이면 다음 방법으로 테스트를 할 수 있습니다.

In [ ]:
!curl -X POST http://127.0.0.1:8888/ \
-H "Content-Type: application/json" \
-H "Accept: text/event-stream" \
-d "{\"messages\":[{\"role\":\"user\",\"content\":\"2+2는 뭘까?\"}]}"

응답이 올바로 나왔다면, Ctrl + C를 눌러서 서버를 종료합니다.

이제, 기존 소스에 날씨 정보를 제공하는 Function 도구와 에이전트를 추가합니다.   
두번째 Agnet는 루트(/) 경로가 아닌 /weather 경로로 노출합니다.   
이런 방식으로 서로 다른 Agent를 각각 서로 다른 경로로 노출할 수 있습니다.  

아래의 코드가 적용된 파일은 agent_server2.py로 이미 현재 폴더에서 제공되고 있습니다.

In [ ]:
# 기존 소스에 아래 부분을 추가
# -----------------------------------------------------
# 날씨 정보를 제공하는 도구 함수 정의
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C."

# 날씨 정보를 제공하는 Agent 생성
weather_agent = ChatAgent(
    name="AGUIAssistant",
    instructions="You are a helpful assistant.",
    chat_client=chat_client,
    tools=[get_weather]
)
# -----------------------------------------------------
# 기존 소스에 여기까 추가합니다. 

# Create FastAPI app
app = FastAPI(title="AG-UI Server")

# Register the AG-UI endpoint
add_agent_framework_fastapi_endpoint(app, agent, "/")
# 기존 소스에 weather_agent 등록 부분 추가
add_agent_framework_fastapi_endpoint(app, weather_agent, "/weather")  

#... 기존 소스 ...

터미널을 새로 열고 다음과 같이 명령을 실행하여 서버를 시작합니다.

``` powershell
    python agent_server2.py
```

서버가 실행 중이면 다음 방법으로 테스트를 할 수 있습니다.

In [ ]:
!curl -X POST http://127.0.0.1:8888/weather \
-H "Content-Type: application/json" \
-d "{\"messages\":[{\"role\":\"user\",\"content\":\"오늘 서울의 날씨는?\"}]}"

다음은 클라이언트를 만들어 봅니다.